# Import Libs

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" #If the line below doesn't work, uncomment this line (make sure to comment the line below); it should help.
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
'''
Numbers for "os.environ['TF_CPP_MIN_LOG_LEVEL']": 
0 = all messages are logged (default behavior)
1 = INFO messages are not printed
2 = INFO and WARNING messages are not printed
3 = INFO, WARNING, and ERROR messages are not printed
'''

import tensorflow
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Ignore Pandas warnings of deprecation
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' | Disable warnings
import numpy as np
from time import time
from keras.layers import Dense, Dropout, SimpleRNN, RNN, LSTM, GRU
from keras import Sequential, models
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from termcolor import colored
!pip install tensorflow_model_optimization
import tensorflow_model_optimization as tfmot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import other Python files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
!cp -r "/content/drive/My Drive/Colab Notebooks/unsw_processing.ipynb" '/content/'
!cp -r "/content/drive/My Drive/Colab Notebooks/results_visualization.ipynb" '/content/'
!cp -r "/content/drive/My Drive/Colab Notebooks/quant_model" '/content'

!pip install import_ipynb
import import_ipynb

from unsw_processing import unsw_encoding
from results_visualization import print_results


print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))
print(tensorflow.test.gpu_device_name())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Num GPUs Available:  0



# Training

In [ ]:
csv_values = ['epochs', 'acc', 'loss', 'val_acc', 'val_loss',
              'loss_fct', 'optimizer', 'activation_fct',
              'layer_nb', 'unit_nb', 'batch_size', 'dropout', 'cell_type',
              'encoder']

csv_best_res = ['param', 'value', 'min_mean_val_loss']

train_params = {'epochs': 100, 
          'loss_fct': 'mse', 
          'optimizer': 'rmsprop',
          'activation_fct': 'tanh', 
          'layer_nb': 1, 'unit_nb': 128, 
          'batch_size': 1024, 'dropout': 0.4,
          'encoder': 'labelencoder',
          'shuffle': True}

# ***** VARIABLE PARAMETERS *****
# 'encoder': ['standardscaler', 'labelencoder', 'minmaxscaler01', 'minmaxscaler11', 'ordinalencoder'],
# 'optimizer': ['adam', 'sgd', 'rmsprop', 'nadam', 'adamax', 'adadelta'],
# 'activation_fct': ['sigmoid', 'softmax', 'relu', 'tanh']
# 'layer_nb': [1, 2, 3, 4]
# 'unit_nb': [4, 8, 32, 64, 128, 256]
# 'dropout': [0.1, 0.2, 0.3, 0.4]
# 'batch_size': [512, 1024, 2048]

tf_model_path = '/content/content/tf_models'
res_path = "/content/drive/My Drive/Colab Notebooks/quant_results/"

# Name for result files
res_name = train_params['loss_fct'] + '_' + train_params['optimizer'] + '_' +\
        train_params['activation_fct'] + '_' + str(train_params['layer_nb']) + '_' +\
        str(train_params['unit_nb']) + '_' + str(train_params['batch_size']) + '_' +\
        str(train_params['dropout']) + '_' + "RNN" + '_' +\
        train_params['encoder'] + '_' + str(time())

In [ ]:
# Encode dataset and return : x_train, x_test, y_train, y_tests
def load_data():
    x_train, x_test, y_train, y_test = unsw_encoding(train_params)

    # Reshape the inputs in the accepted model format
    x_train = np.array(x_train).reshape([-1, x_train.shape[1], 1])
    x_test = np.array(x_test).reshape([-1, x_test.shape[1], 1])
    return x_train, x_test, y_train, y_test

In [ ]:
# Create and train a model
def train_model(x_train, x_test, y_train, y_test):
    
    cell = SimpleRNN #RNN

    if os.path.exists('/content/quant_model'):
        save_model = ModelCheckpoint(filepath=tf_model_path + res_name,
                                        monitor='val_accuracy', save_best_only=True)
        callbacks = [save_model]
        loaded_model = models.load_model('/content/quant_model')
        loaded_model.compile(loss=train_params['loss_fct'], optimizer=train_params['optimizer'],
                      metrics=['accuracy'])
        loaded_model.fit(x_train, y_train, train_params['epochs'],
                        verbose=1, shuffle=train_params['shuffle'],
                        validation_data=(x_test, y_test), callbacks=callbacks)

        
        # Save model to path
        loaded_model.save(tf_model_path)

        # Train model and get important values for saving results
        hist = loaded_model.fit(x_train, y_train, train_params['batch_size'], train_params['epochs'],
                        verbose=1, shuffle=train_params['shuffle'],
                        validation_data=(x_test, y_test), callbacks=callbacks)

        
        # Print accuracy/loss per epoch
        print_results(train_params, loaded_model, x_train, x_test, y_train, y_test)

        return hist

    else:
        print("NO QUANTIZED MODEL FOUND!")

In [ ]:
if __name__ == "__main__":
    print("Loading Data")
    x_train, x_test, y_train, y_test = load_data()
    print("Training Model")
    history = train_model(x_train, x_test, y_train, y_test)
    

    # --- SAVE RESULTS --- #

    epochs = len(history.history['accuracy'])
    df = pd.DataFrame(columns=csv_values)

    try:
      for epoch in range(epochs):
        df = df.append({'epochs': epoch,
                        'acc':  history.history['accuracy'][epoch],
                        'loss': history.history['loss'][epoch],
                        'val_acc': history.history['val_accuracy'][epoch],
                        'val_loss': history.history['val_loss'][epoch],
                        'loss_fct': train_params['loss_fct'],
                        'optimizer': train_params['optimizer'],
                        'activation_fct': train_params['activation_fct'],
                        'layer_nb': train_params['layer_nb'],
                        'unit_nb': train_params['unit_nb'],
                        'batch_size': train_params['batch_size'],
                        'dropout': train_params['dropout'],
                        'cell_type': "RNN",
                        'encoder': train_params['encoder']},
                    ignore_index=True)
    except Exception as e:
      print("Could not append to df due to {}".format(colored(e, 'red')))
    
    res_name = train_params['loss_fct'] + '_' + train_params['optimizer'] + '_' +\
        train_params['activation_fct'] + '_' + str(train_params['layer_nb']) + '_' +\
        str(train_params['unit_nb']) + '_' + str(train_params['batch_size']) + '_' +\
        str(train_params['dropout']) + '_' + "RNN" + '_' +\
        train_params['encoder'] + '_' + str(time())
        
    # Errors occur without this
    if not os.path.exists(res_path):
        os.makedirs(res_path)
  
    full_res_path = res_path + "quant_model_full_results_" + res_name + ".csv"
    df.to_csv(full_res_path, index=False)

Loading Data
Training Model


821/824 [============================>.] - ETA: 0s - loss: 0.0220 - accuracy: 0.8520

824/824 [==============================] - 38s 45ms/step - loss: 0.0220 - accuracy: 0.8519 - val_loss: 0.0387 - val_accuracy: 0.7300


Epoch 1/100
81/81 [==============================] - ETA: 0s - loss: 0.0204 - accuracy: 0.8628

81/81 [==============================] - 17s 212ms/step - loss: 0.0204 - accuracy: 0.8628 - val_loss: 0.0350 - val_accuracy: 0.7479
Epoch 2/100
81/81 [==============================] - 16s 194ms/step - loss: 0.0202 - accuracy: 0.8636 - val_loss: 0.0346 - val_accuracy: 0.7470
Epoch 3/100
81/81 [==============================] - ETA: 0s - loss: 0.0201 - accuracy: 0.8652

...

81/81 [==============================] - 17s 210ms/step - loss: 0.0200 - accuracy: 0.8651 - val_loss: 0.0358 - val_accuracy: 0.7552
Epoch 5/100
81/81 [==============================] - 16s 196ms/step - loss: 0.0199 - accuracy: 0.8665 - val_loss: 0.0363 - val_accuracy: 0.7401
Epoch 6/100
81/81 [==============================] - 17s 213ms/step - loss: 0.0199 - accuracy: 0.8652 - val_loss: 0.0353 - val_accuracy: 0.7441
Epoch 7/100
81/81 [==============================] - 16s 194ms/step - loss: 0.0199 - accuracy: 0.8655 - val_loss: 0.0364 - val_accuracy: 0.7402
Epoch 8/100
81/81 [==============================] - 16s 195ms/step - loss: 0.0198 - accuracy: 0.8663 - val_loss: 0.0359 - val_accuracy: 0.7481
Epoch 9/100
81/81 [==============================] - 16s 197ms/step - loss: 0.0198 - accuracy: 0.8663 - val_loss: 0.0359 - val_accuracy: 0.7435
Epoch 10/100
81/81 [==============================] - 15s 190ms/step - loss: 0.0199 - accuracy: 0.8666 - val_loss: 0.0358 - val_accuracy: 0.7394
Epo

81/81 [==============================] - 17s 213ms/step - loss: 0.0196 - accuracy: 0.8683 - val_loss: 0.0346 - val_accuracy: 0.7560
Epoch 29/100
81/81 [==============================] - ETA: 0s - loss: 0.0195 - accuracy: 0.8668

81/81 [==============================] - 17s 213ms/step - loss: 0.0195 - accuracy: 0.8668 - val_loss: 0.0353 - val_accuracy: 0.7589
Epoch 30/100
81/81 [==============================] - 16s 194ms/step - loss: 0.0195 - accuracy: 0.8681 - val_loss: 0.0363 - val_accuracy: 0.7523
Epoch 31/100
81/81 [==============================] - 17s 217ms/step - loss: 0.0195 - accuracy: 0.8681 - val_loss: 0.0363 - val_accuracy: 0.7417
Epoch 32/100
81/81 [==============================] - 16s 195ms/step - loss: 0.0195 - accuracy: 0.8678 - val_loss: 0.0350 - val_accuracy: 0.7442
Epoch 33/100
81/81 [==============================] - 16s 202ms/step - loss: 0.0195 - accuracy: 0.8680 - val_loss: 0.0375 - val_accuracy: 0.7342
Epoch 34/100
81/81 [==============================] - 16s 193ms/step - loss: 0.0195 - accuracy: 0.8671 - val_loss: 0.0347 - val_accuracy: 0.7479
Epoch 35/100
81/81 [==============================] - 16s 193ms/step - loss: 0.0195 - accuracy: 0.8683 - val_loss: 0.0381 - val_accuracy: 0.739

81/81 [==============================] - 16s 200ms/step - loss: 0.0192 - accuracy: 0.8701 - val_loss: 0.0343 - val_accuracy: 0.7614
Epoch 60/100
81/81 [==============================] - 15s 187ms/step - loss: 0.0192 - accuracy: 0.8710 - val_loss: 0.0362 - val_accuracy: 0.7536
Epoch 61/100
81/81 [==============================] - 15s 185ms/step - loss: 0.0192 - accuracy: 0.8703 - val_loss: 0.0369 - val_accuracy: 0.7429
Epoch 62/100
81/81 [==============================] - 15s 185ms/step - loss: 0.0192 - accuracy: 0.8695 - val_loss: 0.0364 - val_accuracy: 0.7437
Epoch 63/100
81/81 [==============================] - 15s 188ms/step - loss: 0.0192 - accuracy: 0.8697 - val_loss: 0.0359 - val_accuracy: 0.7557
Epoch 64/100
81/81 [==============================] - 15s 189ms/step - loss: 0.0193 - accuracy: 0.8695 - val_loss: 0.0395 - val_accuracy: 0.7283
Epoch 65/100
81/81 [==============================] - 16s 193ms/step - loss: 0.0192 - accuracy: 0.8703 - val_loss: 0.0361 - val_accuracy: 0.748

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Save Quantized Model to Drive

In [ ]:
# Check if model is Quantized

import tensorflow
from keras import Sequential, models

loaded_model = models.load_model('/content/quant_model')
print(loaded_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 128)               16640     
                                                                 
 quant_dropout (QuantizeWrap  (None, 128)              1         
 perV2)                                                          
                                                                 
 quant_dense (QuantizeWrappe  (None, 10)               1295      
 rV2)                                                            
                                                                 
Total params: 17,936
Trainable params: 17,930
Non-trainable params: 6
_________________________________________________________________
None


In [ ]:
!cp -r "/content/quant_model" "/content/drive/My Drive/Colab Notebooks/"